In [ ]:
import pandas as pd
import numpy as np
from tvDatafeed import TvDatafeed,Interval
import datetime 
from astral.moon import phase 



# Extrayendo data de tradingview con tvdatafeed

In [ ]:
username = 'labradoradrian770'
password = 'eva210920'

In [ ]:
tv = TvDatafeed(username=username,password=password)

In [ ]:
df_ethbtc=tv.get_hist('ETHBTC','BINANCE', Interval.in_daily,n_bars=5000)

In [ ]:
df_ethbtc.info()

In [ ]:
df_ethbtc 

In [ ]:
data= df_ethbtc
data['datetime']= data.index


data

# cambiando el Datetimeindex por un index numerico incremental

In [ ]:
data['datetime'] = pd.to_datetime(data['datetime'], format = "%Y-%m-%d")
data['datetime'].info()

In [ ]:
data = data.reset_index(drop=True)
data



In [ ]:
data= data.drop(columns='symbol',axis=1)
data

In [ ]:
data['datetime'] = data['datetime'].dt.date


In [ ]:
data

In [ ]:
data.info()

In [ ]:
data['datetime']=pd.to_datetime(data['datetime'])
data['datetime'].info()

In [ ]:
data['moon_phase']=''
data

# Ciclo de moon_phase de cada fecha contenida en el df

In [ ]:
for index, row in data.iterrows():
    date = row["datetime"]
    phase_moon = phase(date)
    data.at[index, "moon_phase"] = phase_moon

data

# Creando name_phase column 


In [ ]:
def imprimir_fase(phase):
    if phase >=0 and phase <7:
        return "🌑 new moon"
    elif phase >= 7 and phase <14 :
        return "🌓 first quarter"
    elif phase >=14 and phase <21:
        return "🌕 full moon"
    elif phase >=21 and phase <28 :
        return "🌗 last quarter"
    else:
        return "Fase lunar desconocida."
    
    
data["name_phase"] = ""

for index, row in data.iterrows():
    phase = row["moon_phase"]
    name_phase = imprimir_fase(phase)
    data.at[index, "name_phase"] = name_phase
    
data

# Creando change column para observar cambios de tendencia y porcentaje de cambio entre los precios de cierre

In [ ]:
data.insert(data.columns.get_loc('close')+1, 'change',  data['close'].pct_change().round(2))

In [ ]:
data

In [ ]:
data.columns

# funtion simple para volatility_index

In [ ]:
def volatilidad_index(row):
    return (row['high'] - row['low']) / (row['close'] - row['open'])

data['volatility_index'] = data.apply(volatilidad_index, axis=1)
data['volatility_index'] = (data['volatility_index'] - data['volatility_index'].min()) / (data['volatility_index'].max() - data['volatility_index'].min()) * 2 - 1
data['volatility_index'] = data['volatility_index'].round(2)
data


# Valores con de volatilidad mas comunes 

In [ ]:
data['volatility_index'] = data['volatility_index'].astype(float)
grupo_vola = data['volatility_index'].groupby(data['volatility_index']).count()
grupo_vola

# Aplicando describe veo que aun falta transformar los datos de las columns moon_phase y volume


In [ ]:
data.describe()

In [ ]:
data

In [ ]:
data[['volume','moon_phase']].info()

In [ ]:
data['moon_phase'] = data['moon_phase'].astype('float')

In [ ]:
data=data.round(8)

In [ ]:
data.describe()

# uso de set_option para agregar decimales a los datos y asi evitar outputs en scientific notation

In [ ]:
pd.set_option('display.float_format', lambda x: '%.6f' % x)

In [ ]:
data

In [ ]:
data.describe()

In [ ]:
data

In [ ]:
data[['change']].head(21)

# Funcion para indentificar tendencias,retrocesos y cambios utilizando retrocesos fibonacci

In [ ]:
change_values = data['change'].tolist()

In [ ]:
def trend_identifier(change_values, fibonacci_levels=[0.382, 0.5, 0.618, 0.786, 1]):
    current_trend = None
    trend_start = 0
    trends = []
    for i, change_value in enumerate(change_values[1:]):
        if current_trend is None:
            current_trend = change_value
            trend_start = i + 1
        elif (current_trend > 0 and change_value < 0) or (current_trend < 0 and change_value > 0):
            if abs(change_value) > abs(current_trend) * fibonacci_levels[0]:
                for fibonacci_level in fibonacci_levels[1:]:
                    if abs(change_value) > abs(current_trend) * fibonacci_level:
                        current_trend = change_value
                        trend_start = i + 1
                        break
                else:
                    trends.append((trend_start, i + 1, current_trend))
                    current_trend = None
            else:
                trends.append((trend_start, i + 1, current_trend))
                current_trend = None
        else:
            current_trend += change_value
    if current_trend is not None:
        trends.append((trend_start, len(change_values), current_trend))
    return trends



In [ ]:
trends = trend_identifier(change_values)

In [ ]:
trends

# crear dataframe con los resultados de iterar a la columna change con la funcion trend_identifier

In [ ]:
result = trend_identifier(change_values)

In [ ]:
df_trends= pd.DataFrame(result, columns=['start', 'end', 'trend'])

In [ ]:
df_trends

# para observar cuanto duran las tendencias y cual es el promedio de duracion 

In [ ]:
duration_trend = df_trends['end'] - df_trends['start']
average_duration = duration_values.mean()


In [ ]:
average_duration

In [ ]:
df_trends['duration_trend']= duration_trend

In [ ]:
df_trends

# add moon_phase a al dataframe de tendencias

In [ ]:
df_trends["moon_phase"] = ''

for i, row in df_trends.iterrows():
    start, end = row["start"], row["end"]
    moon_phase_slice = data.iloc[start:end]["moon_phase"]
    df_trends.at[i, "moon_phase"] = moon_phase_slice.mean()

In [ ]:
df_trends